In [ ]:
# 找到array的非零元素
import numpy as np

a = np.array([[1, 2, 0],
              [0, 4, 0],
              [1, 0, 0]])
idxs = np.where(a!=0)   # idxs = ([0, 0, 1, 2],[0, 1, 1, 0]) 手动也行^_^

# 行列数组索引
print(idxs)
print(a[idxs])

# 对应位置布尔类型索引
print(a!=0)
print(a[a!=0])

# 行列数组索引
print(a.nonzero())
print(a[a.nonzero()])

# 行列数组索引
print(np.nonzero(a))
print(a[np.nonzero(a)])

# 对应位置布尔类型索引
print(a!=0)
print(a[a!=0])

# 行列数组索引
print(a.nonzero())
print(a[a.nonzero()])

# 行列数组索引
print(np.nonzero(a))
print(a[np.nonzero(a)])

In [ ]:
# 把np.where()返回的行列号分开的结果合并

a = np.array([[1, 2, 0],
              [0, 4, 0],
              [1, 0, 0]])
print("np.where(a)的初始返回结果：", np.where(a))

# ① 采用xzh的方案
def tuple2list(t):
    return [[t[0][idx], t[1][idx]] for idx in range(len(t[0]))]
rc = tuple2list(np.where(a))
print(rc)

# ② 使用zip
rc = []
for r, c in zip(*np.where(a)):  # 等价于for r, c in zip(np.where(a)[0], np.where(a)[1])
    rc.append([r, c])
print(rc)

# ③ 使用np的堆叠函数把两个分开的数组堆叠起来成为一个矩阵方便索引
rc = np.column_stack(np.where(a))
xy = rc[:, ::-1]
print(rc)
print(xy)

In [ ]:
# 数组元素的排序与挑选
a = np.array([[7, 8, 0],
              [0, 9, 0],
              [7, 0, 0]])
print(np.argsort(a, axis=0))    # 数组元素沿指定轴排序，返回值在指定轴上的索引矩阵
print()

# print(np.sort(a, axis=0))    # 数组元素沿指定轴排序，返回具体值矩阵
# print()

print(np.take_along_axis(a, np.argsort(a, axis=0), axis=0))  # 根据返回的值的索引沿指定轴挑选元素（重排）
print()
b = np.zeros_like(a)
for j in range(a.shape[1]):
    b[:, j] = a[:, j][np.argsort(a, axis=0)[:,j]]
print(b)
print()


# # 只有一个轴式可以不使用take_along_axis函数
# a = np.array([7, 8, 0])
# print(np.argsort(a, axis=0))    # 数组元素沿指定轴排序，返回值在指定轴上的索引矩阵
# print()
# print(np.take_along_axis(a,np.argsort(a, axis=0), axis=0))
# print()

In [ ]:
transform = lambda x: [i[::-1] for i in x]
a = [[1, 2], [3, 4]]
b = transform(a)
b

In [ ]:
a = np.linspace(0, 1, num=4+1, endpoint=False)
a

In [ ]:
points = np.array([0, 1, 2, 3, 4, 5])
print(points[[0, 1, 4]])
print(points[[0, 1, 2, 3, 4, 5]])
print(points[[0, 1, 1, 1, 1, 1]])
print(points[np.bool_([0, 1, 1, 1, 1, 1])])

In [23]:
# GTE encoding
import numpy as np
import pickle
import math
import os.path as osp
from multiprocessing import Pool    # 多进程
from concurrent.futures import ThreadPoolExecutor   # 多线程
# .p文件是rc坐标 

IMG_SIZE = 2048
OFFSET = 1
MAX_DEGREE = 6
VECTOR_NORM = 25.0

NUM_PROCESS = 10
NUM_THREAD = 10
 
# 将图编码为19维张量
graph_pattern = './cityscale/20cities/region_{}_refine_gt_graph.p'
rgb_pattern   = './cityscale/20cities/region_{}_sat.png'
# sat_pattern   = './cityscale/20cities/region_{}_sat.png'
# samplepoints_path = './cityscale/20cities/region_7_refine_gt_graph_samplepoints.json'

save_dir = './cityscale/GTE'


def encode_GTE(tile_id):
    graph_path = graph_pattern.format(tile_id)
    graph_adj = pickle.load(open(graph_path, 'rb'))
    
    GTE = np.zeros((IMG_SIZE, IMG_SIZE, 3*MAX_DEGREE+1))
    for v, neis in graph_adj.items():
        r, c = v
        if (r < 16) or (r > IMG_SIZE- 16) or (c < 16) or (c > IMG_SIZE- 16):
            continue
        for row in range(r-OFFSET, r+OFFSET+1):
            for col in range(c-OFFSET, c+OFFSET+1):
                GTE[row, col, 0] = 1
        
        for nei in neis:
            nei_r, nei_c = nei
            if (nei_r < 16) or (nei_r > IMG_SIZE- 16) or (nei_c < 16) or (nei_c > IMG_SIZE- 16):
                continue
            dx, dy = nei_c-c, nei_r-r
            d = math.atan2(dy, dx) + math.pi
            j = int(d/(math.pi/3.0)) % MAX_DEGREE  # 扇区编号0-5
            
            for row in range(r-OFFSET, r+OFFSET+1):
                for col in range(c-OFFSET, c+OFFSET+1):
                    GTE[row, col, 1+3*j] = 1
                    GTE[row, col, 1+3*j+1] = dx / VECTOR_NORM
                    GTE[row, col, 1+3*j+2] = dy / VECTOR_NORM
                    
    np.savez(osp.join(save_dir, f'region_{tile_id}_GTE.npz'), GTE=GTE)
    
    return  GTE

In [4]:
all_tile_names = [i for i in range(180)]

# 多进程执行任务
# pool = Pool(processes=10)
# pool.map(encode_GTE, all_tile_names)
# pool.close()
# pool.join()

with ThreadPoolExecutor(max_workers=NUM_THREAD) as executor:
    # 使用线程池执行任务
    executor.map(encode_GTE, all_tile_names)

In [56]:
# Decoding for verifying sake
import cv2 as cv
from skimage import measure
from scipy.ndimage import rotate
import scipy


def vis_GT_GTE(GTE, keypoint_thr=0.5, edge_thr=0.5, aug=False, rot_angle=90, rot_index=0):
    # vis_output = np.zeros((512, 512, 3))    # 不加底图
    vis_output = cv.imread(rgb_pattern.format(7))[:512, :512, :]
    sub_GTE = GTE[:512, :512, :]
    
    if aug:
        sub_GTE = GTE[:512, :512, :]
        # prob_GTE = np.concatenate([sub_GTE[:, :, 0:1], sub_GTE[:, :, 1::3]], axis=2)
        # prob_GTE = rotate(prob_GTE, rot_angle*rot_index, axes=(0, 1), reshape=False)
        # sub_GTE[:, :, 0] = prob_GTE[:, :, 0]
        # sub_GTE[:, :, 1::3] = prob_GTE[:, :, 1:]
        # 其中的dx和dy还需要进行更新，因为位置变了，dx, dy也得变
        # rot_radians = math.radians(rot_angle)
        # rot_mat = np.array([
        #         [1, 0, 0],
        #         [0, math.cos(rot_radians), +math.sin(rot_radians)],
        #         [0, -math.sin(rot_radians), math.cos(rot_radians)]
        # ], dtype=np.float32)        # 注意这里的旋转矩阵，由于编码的时候是用的图像坐标，y方向跟笛卡尔坐标是相反的，所以笛卡尔坐标旋转的逆时针对应图像坐标的顺时针旋转
        rot_mat = np.array([    # 只针对90度的旋转，如果是任意角度还需要改旋转矩阵
                        [0, 1],
                        [-1, 0]
            ], dtype=np.float32)
        # rot_mat = np.array([    # 只针对90度的旋转，如果是任意角度还需要改旋转矩阵
        #                 [1, 0, 0],
        #                 [0, 0, 1],
        #                 [0, -1, 0]
        #     ], dtype=np.float32)
        sub_GTE = rotate(sub_GTE, rot_angle*rot_index, axes=(0, 1), reshape=False)
        vis_output = rotate(vis_output, rot_angle*rot_index, axes=(0, 1), reshape=False)
        
        # 只对所有不为0的进行变换，其余的全是0，没必要再计算了
        for r, c in np.column_stack(np.where(sub_GTE[:,:, 0]>0.98)):
            delta_coords = []
            for j in range(MAX_DEGREE):
                delta_coords.append([sub_GTE[r, c, 1+3*j+1], sub_GTE[r, c, 1+3*j+2]])   # dx, dy
            delta_coords_to_rot = np.column_stack(delta_coords)
            roted_coords = np.linalg.matrix_power(rot_mat, rot_index)@delta_coords_to_rot
            
            for j in range(MAX_DEGREE):    # stick back
                sub_GTE[r, c, 1+3*j+1], sub_GTE[r, c, 1+3*j+2] = roted_coords[:, j]
                
                
        # non_zero_indices = np.column_stack(np.where(sub_GTE[:,:, 0]>0.98))
        # delta_coords = sub_GTE[non_zero_indices[:, 0], non_zero_indices[:, 1], 1:].reshape(-1, 3, 6)
        # delta_coords = np.linalg.matrix_power(rot_mat, rot_index)@delta_coords
        # sub_GTE[non_zero_indices[:, 0], non_zero_indices[:, 1], 1:] = delta_coords.reshape(-1, 18)
                

        
    keypoint_map = sub_GTE[:,:, 0]
    
    for r, c in np.column_stack(np.where(keypoint_map)):
        for j in range(MAX_DEGREE):
            dx, dy = sub_GTE[r, c, 1+3*j+1], sub_GTE[r, c, 1+3*j+2]
    
    # 从团中寻找中心点
    keypoint_map = (keypoint_map > keypoint_thr).astype(np.uint8)
    labels = measure.label(keypoint_map, connectivity=2)
    props = measure.regionprops(labels)
    min_area = 4
    for region in props:
        if region.area > min_area:
            center = region.centroid[::-1]   # rc-> xy
            center_x, center_y = int(center[0]), int(center[1])
            cv.circle(vis_output, (center_x, center_y), radius=2, color=(0, 0, 255), thickness=-1)
            
            r, c = center_y, center_x
            for j in range(MAX_DEGREE):
                edgeness = sub_GTE[r, c, 1+3*j]
                if edgeness > edge_thr:
                    dx, dy = sub_GTE[r, c, 1+3*j+1], sub_GTE[r, c, 1+3*j+2]
                    # dx, dy = rot_mat@np.array([dx, dy])
                    dst_x, dst_y = int(center_x + VECTOR_NORM*dx), int(center_y + VECTOR_NORM*dy)
                    cv.line(vis_output, (center_x, center_y), (dst_x, dst_y), color=(0, 255, 0), thickness=1)
    cv.imwrite(osp.join(verify_dir, f'region_7_vis_{rot_angle*rot_index}_aug.png'), vis_output)
    
    
    
GTE = encode_GTE(tile_id=7)
verify_dir = './cityscale/verify'
vis_GT_GTE(GTE, aug=True, rot_angle=90, rot_index=6)
    
# def decodeGTE(GTE):
    

In [73]:
import torch

a = torch.normal(mean=0, std=1, size=(2, 2, 3))
print(a)
b = a.permute(2, 0, 1)
pick = torch.ones((3,)).to(torch.bool)
pick[1] = False
print(a)
print(b[pick])

torch.permute()

tensor([[[-0.5897, -0.9032, -0.1924],
         [ 1.0630, -0.1089,  1.2133]],

        [[-0.7930, -0.1975, -0.2154],
         [ 1.2723,  0.1409, -0.4620]]])
tensor([[[-0.5897, -0.9032, -0.1924],
         [ 1.0630, -0.1089,  1.2133]],

        [[-0.7930, -0.1975, -0.2154],
         [ 1.2723,  0.1409, -0.4620]]])
tensor([[[-0.5897,  1.0630],
         [-0.7930,  1.2723]],

        [[-0.1924,  1.2133],
         [-0.2154, -0.4620]]])
